In [ ]:
# standard imports for blockchain transactions
import os
import bit
import json
import requests
import subprocess
from bit import wif_to_key
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account
from constants import *
from pathlib import Path
from getpass import getpass
import warnings
warnings.filterwarnings("ignore")


In [ ]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [ ]:
# use this function in case you want to use the key pairs store in env file
def get_priv_key(coin):
    
    """fetches the private key from environment variable or keystore file  depending on the cointype.
    params: cointype e.g ETH or BTC as string
    output: private key as string"""
    
    load_dotenv()
    if coin.upper() =='ETH':
         priv_key = os.getenv('ETH_PRIVATE_KEY')
    elif coin.upper() == 'BTCTEST':
        priv_key = os.getenv('BTC_PRIVATE_KEY')
    return priv_key
    

# use this function to dynamically fetch the key pairs using hd-wallet-derive
def key_derive(mnemonic, cointype, numderive, cols):
    
    """function invoces hd-wallet-derive to fetch the key pairs using BIP44 based on BIP39 tree of keys 
    from the master key generated by bip32 mnemonic phase.
    params: mnemonic phrase as string, cointype e.g: ETH or BTCTEST as string, 
            numderive - number of key pairs to derive as int
            cols - columns of the bip44 output as string
    output: dictionary of key objects with for each associated cointype"""
    
    keys={}
    if type(cointype)=='str':
        cointype=[cointype]
    for coin in cointype: 
        command = 'php derive -g --mnemonic=' 
        command = command + '"' + mnemonic + '"'
        command = command + ' --coin=' + coin + ' --numderive=' +f'{numderive}' 
        command = command + ' --cols=' + cols + ' --format=json'
        #print(command)
        p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
        (output, err) = p.communicate()
        p_status = p.wait()
        keys[coin] = json.loads(output)
    return keys

     
def priv_key_to_account(priv_key, coin):
    
    """fethces the account address using the private key using either bit or web3 libraries depending on cointype
    params: private key and coin (e.g: ETH, BTC) as string
    output: account object"""
    #print(priv_key)
    if coin.upper() == 'ETH':
        account =  Account.from_key(priv_key)
    elif coin.upper() == 'BTCTEST':
        account = wif_to_key(priv_key)
        #account = key.address
    return account


def create_tx(coin, account, to, amount):
    
    """creates crypto transaction by invoking bit or web3 libraries.
    params: coin - conitype as string, account - sender account object generated via priv_key_to_account function
            to - recipient's account object and amount to transact
    returns raw transaction created by bit or web3"""
    
    if coin.upper() == 'ETH':   
        if w3.eth.getBalance(account.address) > amount * 1.125 : 
            gasEstimate = w3.eth.estimateGas(
                    {"from": account.address, "to": to, "value": int(amount)}
                )

            return {
                'from': account.address,
                'to': to,
                'value': int(amount),
                'gas': gasEstimate,
                'gasPrice': w3.eth.gasPrice,
                'nonce': w3.eth.getTransactionCount(account.address),
                #'chain': w3.eth.chain_id()
            }
        else:
            print("Senders account has INSUFFICIENT funds to make this transaction inlcuding the transaction fees")
            return coin
            
    elif coin.upper() == 'BTCTEST':
        balance = float(account.get_balance('btc'))
        min_balance=float(amount) * 52.5
        if balance < min_balance:
            print("Senders account has INSUFFICIENT funds to make this transaction inlcuding the transaction fees")
            return coin
        else:
            return account.prepare_transaction(account.address, [(to, amount, BTC)])

        
def send_tx(coin, account, to, amount):

    """functionn invokes creating raw transaction and signs the transaction before broadcasting to the network
    params: coin - cointype e.g eth or bTCtest as string, senders account object, recipients account object and 
    amount to transact
    returns the transaction hash"""
    
    raw_tx = create_tx(coin, sender, recipient, amount)
    if raw_tx == coin :
        result = "Failed transaction due to insufficient funds"
        
    else:
        if coin.upper() == 'ETH':
            signed_tx = account.sign_transaction(raw_tx)
            result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
            result = w3.eth.getTransaction(result.hex())
        elif coin.upper() == 'BTCTEST': 
            signed_tx = account.sign_transaction(raw_tx)
            result = bit.network.NetworkAPI.broadcast_tx_testnet(signed_tx)
   
    return result


def get_transaction_status(txhash, token):

    """This is a standalone function to check the transaction status
    params: accepts transaction hash and the coin type (ETH/BTCTEST) as strings.
    returns the transaction status with blocknumber and Transaction ID once confirmed"""

    if token.upper() == 'ETH': 
        tx= w3.eth.getTransaction(txhash)
        if tx.blockNumber == None:
            print("The Transaction To : " f'{tx.to}' + "of Value : " + f'{tx.value}' + " with a gasPrice of : "+  f'{tx.gasPrice}' +  
            " with transaction hash : " + f'{txhash}' + " is PENDING to be Mined." 
            )
        else:
            print("The Transaction To : " f'{tx.to}' + " of Value : " + f'{tx.value}' + " with a gasPrice of : " + f'{tx.gasPrice}' +  
            " with transaction hash : " + f'{txhash}' + " has been SUCCESSFUL. The Block Number is : " + f'{tx.blockNumber}' +
            " transaction index is : " + f'{tx.transactionIndex}')  
    
    elif token.upper() == 'BTCTEST': 
        request_url = "https://api.bitaps.com/btc/testnet/v1/blockchain/transaction/" + txhash
        response_data = requests.get(request_url).json()
        tx_data=response_data['data']
        utxo=(tx_data['inputsAmount'] - tx_data['fee'] - tx_data['vOut']['0']['value'])/10**8
        
        print("Your Transaction to the recipient : "+ tx_data['vOut']['0']['address'] + 
            " of value " + f'{tx_data["vOut"]["0"]["value"]/10**8}' + 
            " BTC has " + f'{tx_data["confirmations"]}' + " confirmations. \n " +
            "Transaction BlockIndex is " + f'{tx_data["blockIndex"]}' + " with Blocknumber " + f'{tx_data["blockHeight"]}' + 
            " \nYour Unspent Transaction Output is : " + f'{utxo}') 
            

def get_balance(sender, token):
    """This is a standalone function to check Account balances
    params: accepts sender account as an object. run the priv_key_to_account(privkey, coin) function to get an account object
    returns the account balance"""

    if token.upper() == 'ETH':
        print("Balance in the Sender's account :" +  f'{w3.eth.getBalance(sender.address)}')
      
    else:
        print("Balance in the Sender's account :" + f'{sender.get_balance("btc")}')
        print("Unpent Transaction Ouput : " + f'{sender.get_unspents()[0].amount}')


In [ ]:
#configure bip39 to get account keys
cointype=[BTCTEST, ETH, BTC]
mnemonic=input("Enter 12 word Mnenomic : ")
numderive =input("Enter number of child account keys to derive : ")
cols='address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub'
coins = key_derive(mnemonic, cointype, numderive, cols)

print("Private key of the first BTC-Test child node in bip44 tree : ")
print(coins['btc-test'][0]['privkey'])
print("You can access your token keys using the command : coins[token][index# of the key]['privkey'] \n")

In [ ]:
coins['eth'][0]['privkey']

In [ ]:
coins['eth']

In [ ]:
#configure transaction parameters
token=input("Enter the token type - ETH | BTCTEST: ")
privkey=input("Enter the private key of your pre-funded account. if you have the key in an envfile enter 'ENV' otherwise enter the privkey : \n") 
if privkey.upper() == 'ENV':
    privkey = get_priv_key(token)
    

sender = priv_key_to_account(privkey, token)
recipient=input("Enter the address of recipient account : ")
amount = input("enter the amount of tokens you want to transact. If ETH, enter higher value, if BTCtest enter smallest fraction : ")    
send_coin = send_tx(token, sender, recipient, int(amount))

In [ ]:
send_coin

In [ ]:
get_transaction_status('0x756802f260a4868ceccc240c1724ecb85417598f8bdb3a81d9d2bd668d6eefdc', 'ETH')

In [ ]:
get_balance(sender_acnt, 'ETH')

In [ ]:
tx_hash= "3e293d360b602f2f0b156f55e9fd07959dc264a4f2febef2b8973a2cbf69cb5b"
get_transaction_status(tx_hash, 'BTCTEST')